# Cython live coding demo

First, we must import libraries and enable cython in cells.

In [1]:
import numpy as np

# This enables cython in the notebook cells
%load_ext cython

# This installs and enables the line profiler
!pip install line-profiler
%load_ext line_profiler

## Matrix multiplication

First example: matrix multiplication.

The following simply defines the dimensions and some random matrices A and B. We need this to have something to work with.

In [2]:
n = 100
m = 140
o = 90
A = np.random.uniform(size=(n, m))
B = np.random.uniform(size=(m, o))

C = np.matmul(A, B)
print(C.shape)

(100, 90)


In [0]:
def multiply(A, B):
  ret = np.zeros((A.shape[0], B.shape[1]))
  for i in range(A.shape[0]):
    for j in range(B.shape[1]):
      for k in range(A.shape[1]):
        ret[i, j] += A[i, k]*B[k, j]
  return ret

assert np.all(np.isclose(multiply(A, B)-C, 0))



When we run a profiler, we can see where most of the computer power is spent.

In [0]:
%lprun -f multiply multiply(A, B)

Now for the cython version.

In [5]:
%%cython -a
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def multiply_cy(A, B):
  cdef int i, j, k
  cdef double[:,:] A_memview = A
  cdef double[:,:] B_memview = B
  ret = np.zeros((A.shape[0], B.shape[1]))
  cdef double[:,:] ret_memview = ret
  for i in range(A.shape[0]):
    for j in range(B.shape[1]):
      for k in range(A.shape[1]):
        ret_memview[i, j] += A_memview[i, k]*B_memview[k, j]
  return ret


We can use an aassert statemeent to check that we get the correct result out of the cython code.

In [0]:
assert np.all(np.isclose(multiply_cy(A, B)-C, 0))

Now for benchmarking the different implementations

In [7]:
%timeit np.matmul(A, B)
%timeit multiply(A, B)
%timeit multiply_cy(A, B)

The slowest run took 62.62 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 114 µs per loop
1 loop, best of 3: 1.11 s per loop
100 loops, best of 3: 2.36 ms per loop


Yay! Much faster :D (but always use numpy, it's unlikely that any of us would do a better job than they have at optimisation)

## Prime numbers

We can find all the prime numbers below some limit by running the [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) algorithm.

In [0]:
def primes(max_number = 100):
  ret = list(range(2, max_number))
  n = 2
  while n <= max_number:
    if n in ret:
      for m in range(n*2, max_number, n):
        if m in ret:
          ret.remove(m)
    n += 1
  return ret

Again, we check where to focus using the line profiler.

In [0]:
%lprun -f primes primes(10000)

Note how much time is spent running 'in' and 'remove' operations.

In [10]:
%%cython -a
import numpy as np
cimport cython

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
def primes_cy(int max_number = 100):
  ret = np.ones(max_number+1, dtype=np.intc)
  cdef int[:] ret_memview = ret
  ret[0] = 0
  ret[1] = 0
  cdef int n, m, p
  cdef int o = ret.shape[0]
  for n in range(2, max_number+1):
    if ret_memview[n]:
      for m in range(n*2, o, n):
        if ret_memview[m]:
          ret_memview[m] = 0
    n += 1
  return [n for n in range(ret.shape[0]) if ret[n]]

In [0]:
p1 = primes()
p2 = primes_cy()
assert len(p1)==len(p2)
assert np.all([n==m for n, m in zip(p1, p2)])

In [12]:
%timeit primes(1000)
%timeit primes_cy(1000)

100 loops, best of 3: 9.25 ms per loop
10000 loops, best of 3: 173 µs per loop


Again, way faster.

## Cython as a module

An example of how to use map with cython.

In [13]:
%%writefile _demomodule.pyx
#cython: language_level=3

def _squared_value_cy(value):
  return value*value

Overwriting _demomodule.pyx


In [16]:
# Enable cython imports
import pyximport
pyximport.install()

from _demomodule import _squared_value_cy
print("_squared_value_cy(3):", _squared_value_cy(3))

# The function run by map
def parallel_function(value):
  # Import the function
  from _demomodule import _squared_value_cy
  # Run the function
  return _squared_value_cy(value)

# Imports for multiprocessing
from multiprocessing import Pool
import os

indata = list(range(10000))
# This fork our process and runs the map in several processes
with Pool(processes=os.cpu_count()) as pool:
  outdata = pool.map(parallel_function, indata)

print(indata[:15])
print(outdata[:15])

_squared_value_cy(3) 9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196]
